In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Bidirectional,Input
from tensorflow.keras.layers import BatchNormalization, TimeDistributed,LSTM,Dropout,Reshape
from tensorflow.keras import Model
from tensorflow.keras import Model
import numpy as np 
import pandas as pd
import os
from data import preproc as pp
from matplotlib import pyplot as plt
from PIL import Image
import random

ImportError: libSM.so.6: cannot open shared object file: No such file or directory

In [84]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [85]:
data_dir = '/home/austin/Documents/Github/handwritingnotebook/data/'

In [86]:
data_csv = pd.read_csv(data_dir+'words_csv/2020-06-03 11:39:42.000901.csv')

In [87]:
BATCH_SIZE = 100
IMG_HEIGHT = data_csv['height'].max()
IMG_WIDTH = data_csv['width'].max()
DATASET_SIZE = data_csv.shape[0]
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz.,()?!':; "

In [105]:
IMG_HEIGHT

464

In [107]:
IMG_WIDTH

783

https://www.tensorflow.org/tutorials/load_data/images

In [88]:
#dataset = tf.data.TextLineDataset(data_dir+'words_csv/2020-06-03 11:39:42.000901.csv').skip(1)

In [89]:
def convert_to_onehot(data):
    #Creates a dict, that maps to every char of alphabet an unique int based on position
    char_to_int = dict((c,i) for i,c in enumerate(alphabet))
    encoded_data = []
    #Replaces every char in data with the mapped int
    encoded_data.append([char_to_int[char] for char in data])
    encoded_data = encoded_data[0] # Prints the int encoded array

    #This part now replaces the int by an one-hot array with size alphabet
    letter = [0. for _ in range(len(alphabet))]
    for value in encoded_data:
        #At first, the whole array is initialized with 0
        #Only at the number of the int, 1 is written
        letter[value] = 1.
    letter = tf.convert_to_tensor(letter)
    return letter


In [90]:
def preprocess_image(img_path,data_dir):
    img_path = data_dir + 'words_screenshot_labeled/' + img_path
    img = tf.image.decode_png(tf.io.read_file(img_path),channels = 3)
    img = tf.image.rgb_to_grayscale(img)
    img = tf.image.resize_with_pad(img,IMG_HEIGHT,IMG_WIDTH)
    img = tf.image.transpose(img)
    img = img/255
    return img

In [91]:
def create_img_dataset(dataset,data_dir):
    img_dataset_list = []
    for index, row in dataset.iterrows():
        img_dataset_list.append(preprocess_image(row[0],data_dir))
    data_as_dataset = tf.data.Dataset.from_tensor_slices(img_dataset_list)
    return data_as_dataset

In [92]:
def create_label_dataset(dataset):
    label_dataset_list = []
    for index, row in dataset.iterrows():
        label_dataset_list.append(convert_to_onehot(row[1]))
    data_as_dataset = tf.data.Dataset.from_tensor_slices(label_dataset_list)
    return data_as_dataset

In [93]:
imgs = create_img_dataset(data_csv,data_dir)
labels = create_label_dataset(data_csv)

In [94]:
labeled_ds = tf.data.Dataset.zip((imgs,labels))

In [95]:
train_size = int(0.90 * DATASET_SIZE)
test_size = int(0.10 * DATASET_SIZE)

full_dataset = labeled_ds.shuffle(DATASET_SIZE)
train_ds = full_dataset.take(train_size)
train_ds = train_ds.batch(BATCH_SIZE, drop_remainder=True)
test_ds = full_dataset.skip(train_size)
test_ds = test_ds.batch(BATCH_SIZE, drop_remainder=True)

In [96]:
test_ds

<BatchDataset shapes: ((100, 783, 464, 1), (100, 62)), types: (tf.float32, tf.float32)>

In [131]:
class MyModel(Model):
    def __init__(self,
                 loss_object,
                 optimizer,
                 train_loss,
                 train_metric,
                 test_loss,
                 test_metric):
        '''
            Setting all the variables for our model.
        '''
        super(MyModel, self).__init__()
        self.cnn1 = Conv2D(filters = 32,
                           kernel_size = (7, 7), 
                           padding="same", 
                           activation="relu", 
                           input_shape = (IMG_WIDTH,IMG_HEIGHT,1)
                          )
        self.norm1 = BatchNormalization(axis = 0)
        self.maxp1 = MaxPooling2D(pool_size=(2, 2))
        self.cnn2 = Conv2D(64, (3, 3), padding="same", activation="relu")
        self.norm2 = BatchNormalization()
        self.maxp2 = MaxPooling2D(pool_size=(2, 2))
        self.cnn3 = Conv2D(128, (3, 3), padding="same", activation="relu")
        self.norm3 = BatchNormalization()
        self.maxp3 = MaxPooling2D(pool_size=(2, 2))
        self.cnn4 = Conv2D(256, (3, 3), padding="same", activation="relu")
        self.norm4 = BatchNormalization()
        self.maxp4 = MaxPooling2D(pool_size=(2, 2))
    
        self.loss_object = loss_object
        self.optimizer = optimizer
        self.train_loss = train_loss
        self.train_metric = train_metric
        self.test_loss = test_loss
        self.test_metric = test_metric
    def cnn_model(self, x):
        '''
            Defining the architecture of our model. This is where we run 
            through our whole dataset and return it, when training and 
            testing.
        '''
        x = self.cnn1(x)
        print(x.get_shape())
        x = self.norm1(x)
        print(x.get_shape())
        x = self.maxp1(x)
        print(x.get_shape())
        x = self.cnn2(x)
        print(x.get_shape())
        x = self.norm2(x)
        print(x.get_shape())
        x = self.maxp2(x)
        print(x.get_shape())
        x = self.cnn3(x)
        print(x.get_shape())
        x = self.norm3(x)
        print(x.get_shape())
        x = self.maxp3(x)
        print(x.get_shape())
        x = self.cnn4(x)
        print(x.get_shape())
        x = self.norm4(x)
        print(x.get_shape())
        x = self.maxp4(x)
        print(x.get_shape())
        shape = x.get_shape()
        print(shape)
        blstm = Reshape((shape[1], shape[2] * shape[3]))(x)
        print(blstm.get_shape())
        blstm = Bidirectional(LSTM(units=512, return_sequences=True, dropout=0.5))(blstm)
        print(blstm.get_shape())
        blstm = Bidirectional(LSTM(units=512, return_sequences=True, dropout=0.5))(blstm)
        print(blstm.get_shape())
        blstm = Bidirectional(LSTM(units=512, return_sequences=True, dropout=0.5))(blstm)
        print(blstm.get_shape())
        blstm = Bidirectional(LSTM(units=512, return_sequences=True, dropout=0.5))(blstm)
        print(blstm.get_shape())
        blstm = Bidirectional(LSTM(units=512, return_sequences=False, dropout=0.5))(blstm)
        print(blstm.get_shape())
        blstm = Dropout(rate=0.5)(blstm)
        print(blstm.get_shape())
        output_data = Dense(units=62, activation="softmax")(blstm)
        print(output_data.get_shape())
        return(output_data)
        
        
        
        
    @tf.function
    def train_step(self, images, labels):
        '''
            This is a TensorFlow function, run once for each epoch for the
            whole input. We move forward first, then calculate gradients 
            with Gradient Tape to move backwards.
        '''
        with tf.GradientTape() as tape:
            predictions = self.cnn_model(images)
            loss = self.loss_object(labels, predictions)
        gradients = tape.gradient(loss, self.trainable_variables)
        optimizer.apply_gradients(zip(
                                  gradients, self.trainable_variables))

        self.train_loss(loss)
        self.train_metric(labels, predictions)
        
    def fit(self, train, test, epochs):
        '''
            This fit function runs training and testing.
        '''
        for epoch in range(epochs):
            for images, labels in train:
                self.train_step(images, labels)

            for test_images, test_labels in test:
                self.test_step(test_images, test_labels)

            template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
            print(template.format(epoch+1,
                                  self.train_loss.result(),
                                  self.train_metric.result()*100,
                                  self.test_loss.result(),
                                  self.test_metric.result()*100))

            # Reset the metrics for the next epoch
            self.train_loss.reset_states()
            self.train_metric.reset_states()
            self.test_loss.reset_states()
            self.test_metric.reset_states()

In [143]:
# Make a loss object
#loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
loss_object = tf.keras.losses.CategoricalCrossentropy()
# Select the optimizer
optimizer = tf.keras.optimizers.Adam()

# Specify metrics for training
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# Specify metrics for testing
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [144]:
# Create an instance of the model
model = MyModel(loss_object = loss_object,
                optimizer = optimizer,
                train_loss = train_loss,
                train_metric = train_metric,
                test_loss = test_loss,
                test_metric = test_metric)

EPOCHS = 5

In [145]:
model.fit(train = train_ds,
          test = test_ds,
          epochs = EPOCHS)

(100, 783, 464, 32)
(100, 783, 464, 32)
(100, 391, 232, 32)
(100, 391, 232, 64)
(100, 391, 232, 64)
(100, 195, 116, 64)
(100, 195, 116, 128)
(100, 195, 116, 128)
(100, 97, 58, 128)
(100, 97, 58, 256)
(100, 97, 58, 256)
(100, 48, 29, 256)
(100, 48, 29, 256)
(100, 48, 7424)
(100, 48, 1024)
(100, 48, 1024)
(100, 48, 1024)
(100, 48, 1024)
(100, 1024)
(100, 1024)
(100, 62)


ValueError: in user code:

    <ipython-input-131-aa89965ce1d9>:105 train_step  *
        self.train_metric(labels, predictions)
    /home/austin/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/metrics.py:208 __call__  **
        replica_local_fn, *args, **kwargs)
    /home/austin/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/distribute/distributed_training_utils.py:1133 call_replica_local_fn
        return fn(*args, **kwargs)
    /home/austin/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/metrics.py:188 replica_local_fn
        update_op = self.update_state(*args, **kwargs)  # pylint: disable=not-callable
    /home/austin/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/utils/metrics_utils.py:90 decorated
        update_op = update_state_fn(*args, **kwargs)
    /home/austin/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/metrics.py:603